In [44]:
%store -r

In [45]:
import os
import pathlib
from glob import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import xarray as xr

# Harmonize your data
 Make sure that the grids for each of your layers match up. Check out the ds.rio.reproject_match() method from rioxarray.

In [46]:
# List of DataFrames to loop through
dfs = [soil_df, topo_df, MACAv2_df]

forest_df = dfs[0]

for merging_df in dfs[1:]:
    # Join DataFrames
    forest_df = pd.merge(forest_df, merging_df, on='site_name')

forest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   site_name         4 non-null      object
 1   soil_variable     4 non-null      object
 2   soil_da           4 non-null      object
 3   elevation_da      4 non-null      object
 4   slope_da          4 non-null      object
 5   climate_variable  4 non-null      object
 6   climate_scenario  4 non-null      object
 7   start_year        4 non-null      int64 
 8   climate_da        4 non-null      object
dtypes: int64(1), object(8)
memory usage: 420.0+ bytes


In [47]:
maca_da_repr_list = []

for i in range(len(MACAv2_df)):

    maca_da_orig = (
        MACAv2_df.iloc[i]['climate_da']
        .rio.write_crs(4326)
        .rio.set_spatial_dims('lat', 'lon')
        .groupby('time.year')
        .sum()
        .min('year')
    )
    maca_da_repr = maca_da_orig.rio.reproject_match(topo_df.elevation_da[0])
    maca_da_repr_list.append(maca_da_repr)



In [48]:
MACAv2_df['climate_da'] = maca_da_repr_list

In [49]:
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"sum: {raster.sum().item()}\n"
        f"CRS: {raster.rio.crs}\n"
    )

In [50]:
site_list[0].geometry.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [51]:
print_raster(maca_da)

shape: (6082, 7461)
resolution: (0.0002777777777777779, -0.0002777777777777775)
bounds: (-80.33375, 37.494305555555556, -78.26125, 39.18375)
sum: 48335790080.0
CRS: EPSG:4326



In [52]:
print_raster(soil_df.soil_da.values[1])

shape: (4421, 13272)
resolution: (0.0002777777777751535, -0.00027777777777515666)
bounds: (-83.05333333334227, 36.59444444445772, -79.36666666671043, 37.82250000000168)
sum: 299177632.0
CRS: EPSG:4326



In [53]:
print_raster(topo_df.elevation_da.values[0])

shape: (6082, 7461)
resolution: (0.0002777777777777781, -0.0002777777777777777)
bounds: (-80.33375, 37.494305555555556, -78.26125, 39.18375)
sum: 26534074368.0
CRS: EPSG:4326



In [54]:
print_raster(topo_df.slope_da.values[0])

shape: (6082, 7461)
resolution: (0.0002777777777777779, -0.0002777777777777775)
bounds: (-80.33375, 37.494305555555556, -78.26125, 39.18375)
sum: 145385824.0
CRS: EPSG:4326



In [55]:
print_raster(MACAv2_df.climate_da.values[0])

shape: (6082, 7461)
resolution: (0.0002777777777777779, -0.0002777777777777775)
bounds: (-80.33375, 37.494305555555556, -78.26125, 39.18375)
sum: 53029683200.0
CRS: EPSG:4326

